<a href="https://colab.research.google.com/github/mtpradoc/BigQueryAPI/blob/main/06_JOINING_DATA_Dataset_Stack_Overflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Programmatically connect to BigQuery

In order to use a public dataset in BigQuery we need to programmatically authenticate to the google cloud platform

##1. Authenticate to GCP

In [ ]:
from google.colab import auth
auth.authenticate_user()

Let's specify which project_id we are going to use. It can be any

In [ ]:
project_id = 'hazel-env-310501'

##2. Connect to the BigQuery API

In [ ]:
from google.cloud import bigquery

In [ ]:
client = bigquery.Client(project=project_id)

##3. Access the Dataset (public or private)

Let's put a reference for the dataset and project where the dataset we are going to work with

In [ ]:
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

##4. List your tables

In [ ]:
tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables]

print(list_of_tables)

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


##5. Check table 1

In [ ]:
answers_table_ref = dataset_ref.table("posts_answers")

answers_table = client.get_table(answers_table_ref)

client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,18431468,None,<p>Maybe you want to consider datatables.edit<...,None,None,0,None,2013-08-25 17:00:18.170000+00:00,None,2013-08-25 17:00:18.170000+00:00,NaT,None,NaN,None,1312094,18418641,2,0,None,None
1,18431474,None,<p>Changing the 'mydestination' configuration ...,None,None,0,None,2013-08-25 17:01:02.630000+00:00,None,2013-08-25 17:01:02.630000+00:00,NaT,None,NaN,None,976442,18322311,2,0,None,None
2,18431480,None,<p>You are correct that the error is most like...,None,None,0,None,2013-08-25 17:01:51.043000+00:00,None,2013-08-25 17:10:37.830000+00:00,2013-08-25 17:10:37.830000+00:00,None,2278203.0,None,2278203,18431356,2,0,None,None
3,18431483,None,<p><code>FROM</code> is one of the initial log...,None,None,0,None,2013-08-25 17:02:03.147000+00:00,None,2013-08-25 17:02:03.147000+00:00,NaT,None,NaN,None,1179880,18428303,2,0,None,None
4,18431496,None,<p>Cassandra will never order your columns in ...,None,None,0,None,2013-08-25 17:03:13.143000+00:00,None,2013-08-25 17:03:13.143000+00:00,NaT,None,NaN,None,1061519,15278659,2,0,None,None


##5. Check table 2

In [ ]:
questions_table_ref = dataset_ref.table("posts_questions")

questions_table = client.get_table(questions_table_ref)

client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,66138537,DataExplorer with at least two given tags,"<p>I would to fetch data from Data Explorer, I...",None,0,0,None,2021-02-10 14:11:57.947000+00:00,None,2021-02-10 14:11:57.947000+00:00,None,None,None,None,8384006,None,1,0,dataexplorer,2
1,66229417,Using a single object (dataframe) across multi...,<p>I am hosting a server on a private computer...,None,0,0,None,2021-02-16 17:40:44.097000+00:00,None,2021-02-16 17:40:44.097000+00:00,None,None,None,None,12549160,None,1,0,rstudio-server,2
2,66288134,Can we use rpl and leach routing protocol toge...,<p>as I am new to wsn please can anyone tell m...,None,0,0,None,2021-02-20 04:49:09.760000+00:00,None,2021-02-20 04:49:09.760000+00:00,None,None,None,None,15246800,None,1,-1,routes,2
3,66293452,How to clone dynamic component with button click?,<p>My client requirement is duplication of exi...,None,0,0,None,2021-02-20 15:43:15.133000+00:00,None,2021-02-20 15:43:15.133000+00:00,None,None,None,None,7822211,None,1,0,angular-dynamic-components,2
4,66317042,Can Firebase Admin SDK be used in Angular Karm...,<p>I’ve got an Angular CLI app and am testing ...,None,0,0,None,2021-02-22 13:54:54.547000+00:00,None,2021-02-22 13:54:54.547000+00:00,None,None,None,None,183681,None,1,0,angularfire2|angularfire|firebase-admin,2


##6. Explore your data

Something to have into account:

**posts_answers** has a column called **parent_id** which identifies the ID of the question each answer is responding to. **posts_answers** also has an **owner_user_id** column which specifies the ID of the user who answered the question.

 Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [ ]:
questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config=safe_config)

questions_results = questions_query_job.to_dataframe()

print(questions_results.head())

         id                                              title  owner_user_id
0  66318205  BigQuery Search Ads 360 Data Transfer Service ...     15260531.0
1  66406056             Issue in running plx query in BigQuery     14860880.0
2  66382971  Need to pass Bigquery Array of Struct paramete...      3020641.0
3  66201127                   Bigquery SPLIT() and get Nth row      4774960.0
4  66369621  Facing NoSuchMethodError for HttpRequest setRe...      8017771.0


Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.


In [ ]:
answers_query = """
                SELECT pa.id, pa.body, pa.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS pq
                  INNER JOIN
                  `bigquery-public-data.stackoverflow.posts_answers` AS pa
                  ON pa.parent_id=pq.id
                WHERE pq.tags LIKE '%bigquery%'
                """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=26*(1024**3))
answers_query_job = client.query(answers_query, job_config=safe_config)

answers_results = answers_query_job.to_dataframe()

print(answers_results.head())
                

         id                                               body  owner_user_id
0  29950217  <p>The public data are hosted within the US. F...      2989846.0
1  30019547  <p>You can use the REST API to create or updat...      3953357.0
2  30065665  <p>Copying all the project settings and auth s...      4868543.0
3  30125328  <p>Google has recently released a (BETA) featu...       346561.0
4  30167746  <p>Looking into the <a href="https://cloud.goo...      2417948.0


You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:

user_id - contains the owner_user_id column from the posts_answers table
number_of_answers - contains the number of answers the user has written to "bigquery"-related questions

In [ ]:
bigquery_experts_query = """
                         SELECT pa.owner_user_id AS user_id,
                                COUNT(1) AS number_of_answers
                         FROM
                                `bigquery-public-data.stackoverflow.posts_answers` AS pa
                                INNER JOIN
                                `bigquery-public-data.stackoverflow.posts_questions` AS pq
                                ON pa.parent_id=pq.id
                         WHERE pq.tags LIKE '%bigquery%'
                         GROUP BY user_id
                         """
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config)

bigquery_experts_results = bigquery_experts_query_job.to_dataframe()

print(bigquery_experts_results.head())

     user_id  number_of_answers
0  8974140.0                 30
1  9132848.0                  4
2   101923.0                 43
3   769065.0                  1
4  9192468.0                  2


#7.Build a General Python Function

Convert what you've done to a general function a website could call on the backend to get experts on any topic

In [ ]:
def expert_finder(topic,client):
    '''
    Returns a Dataframe with the user IDs who have written Stack Overflow answers on a topic.
    
    Inputs:
      topic: A string with the topic of interest
      client: A Client object that specified the connection to the Stack Overflow dataset
    
    Outputs:
      results: A Dataframe with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts results shown above.
    '''
    my_query = """
               SELECT pa.owner_user_id AS user_id,
                      COUNT(1) AS number_of_answers
               FROM
                      `bigquery-public-data.stackoverflow.posts_questions` AS pq
                      INNER JOIN
                      `bigquery-public-data.stackoverflow.posts_answers` AS pa
                      ON pq.id = pa.parent_id
               WHERE pq.tags LIKE '%"""+topic+"""%'
               GROUP BY pa.owner_user_id
               """
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
    my_query_job = client.query(my_query,job_config=safe_config)

    results = my_query_job.to_dataframe()

    return results

In [ ]:
expert_finder("bigquery", client)

,user_id,number_of_answers
0,9779432.0,2
1,9036351.0,12
2,1489563.0,1
3,179750.0,1
4,8291949.0,3
...,...,...
4532,3486763.0,1
4533,915022.0,1
4534,994308.0,2
4535,3245368.0,1
